In [1]:
# Import required libraries with updated imports
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_huggingface import HuggingFacePipeline
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
import torch


# Define model ID and cache dir
model_id = "meta-llama/Llama-3.2-1B-Instruct"
cache_dir = "./hf_models"

# Check device availability -this is for my macbook / you can use cuda if on windows
device = "mps" if torch.backends.mps.is_available() else "cpu"
print(f"Using device: {device}")

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(
    model_id,
    cache_dir=cache_dir
)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

model = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype=torch.float16 if device in ["mps", "cuda"] else torch.float32,
    device_map="auto",
    cache_dir=cache_dir
)
# You can change this
text_generation_pipeline = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=256,
    temperature=0.1,
    top_p=0.9,
    repetition_penalty=1,
    return_full_text=False,  # This is important
    clean_up_tokenization_spaces=True
)
 
llm = HuggingFacePipeline(
    pipeline=text_generation_pipeline,
    model_kwargs={"return_full_text": False}
)


Using device: mps


In [2]:
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.prompts import ChatPromptTemplate


json_parser = JsonOutputParser()

targeted_resume_prompt = ChatPromptTemplate.from_messages([
    ("system", """You are an expert resume parser and JSON generator.
    Given a professional's experience description, generate a structured JSON representation
    of their work history. Focus only on the work experience section.

    Output Format Instructions:
    - Generate a JSON array of work experiences
    - Each work experience should include:
      * name: Company/Organization name
      * position: Job title
      * startDate: Start date in YYYY-MM-DD format
      * endDate: End date in YYYY-MM-DD format (use current date if still employed)
      * summary: Brief description of the role
      * highlights: Key achievements or responsibilities

    Ensure the JSON is valid and well-structured."""),
    ("user", "Extract work experience from the following description:\n{context}")
])

work_experience_chain = targeted_resume_prompt | llm | json_parser

input_text = """
John Smith worked at TechCorp as a Senior Software Engineer from 2018 to 2023.
During his tenure, he led the development of a cloud-native microservices platform,
reducing system latency by 40%. He implemented CI/CD pipelines and mentored junior
developers. Prior to TechCorp, he was a Software Engineer at InnovateNow from 2015
to 2018, where he developed machine learning algorithms for predictive analytics.
"""

work_experience = work_experience_chain.invoke({"context": input_text})



In [3]:
# Print the generated work experience
import json
print(json.dumps(work_experience, indent=2))

[
  {
    "name": "TechCorp",
    "position": "Senior Software Engineer",
    "startDate": "2018-01-01",
    "endDate": "2023-12-31",
    "summary": "Senior Software Engineer with expertise in cloud-native microservices and CI/CD pipelines.",
    "highlights": [
      "Reduced system latency by 40% through the implementation of CI/CD pipelines and mentoring junior developers.",
      "Developed machine learning algorithms for predictive analytics at InnovateNow."
    ]
  },
  {
    "name": "InnovateNow",
    "position": "Software Engineer",
    "startDate": "2015-01-01",
    "endDate": "2018-12-31",
    "summary": "Software Engineer with expertise in machine learning and predictive analytics.",
    "highlights": [
      "Developed machine learning algorithms for predictive analytics at InnovateNow.",
      "Contributed to various projects as a Freelance Developer."
    ]
  },
  {}
]
